In [ ]:
#TO DO here: upload anemia_data.zip

from google.colab import files

uploaded = files.upload()

Saving anemia_data.zip to anemia_data.zip


In [ ]:
!unzip anemia_data.zip

Archive:  anemia_data.zip
   creating: anemia_data/
  inflating: anemia_data/.DS_Store   
  inflating: __MACOSX/anemia_data/._.DS_Store  
   creating: anemia_data/Images_left_eye/
  inflating: __MACOSX/anemia_data/._Images_left_eye  
  inflating: anemia_data/Anemia_dataset.xlsx  
  inflating: __MACOSX/anemia_data/._Anemia_dataset.xlsx  
   creating: anemia_data/mask_eyes/
  inflating: __MACOSX/anemia_data/._mask_eyes  
  inflating: anemia_data/Images_left_eye/1709880958803.jpg  
  inflating: __MACOSX/anemia_data/Images_left_eye/._1709880958803.jpg  
  inflating: anemia_data/Images_left_eye/1710051336555.jpg  
  inflating: __MACOSX/anemia_data/Images_left_eye/._1710051336555.jpg  
  inflating: anemia_data/Images_left_eye/1709726773680.jpg  
  inflating: __MACOSX/anemia_data/Images_left_eye/._1709726773680.jpg  
  inflating: anemia_data/Images_left_eye/1709984108041.jpg  
  inflating: __MACOSX/anemia_data/Images_left_eye/._1709984108041.jpg  
  inflating: anemia_data/Images_left_eye/1709

In [ ]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

eyes_paths_df = pd.read_excel('/content/anemia_data/Anemia_dataset.xlsx', sheet_name=6)
eyes_paths_df = eyes_paths_df.drop(columns=['Unnamed: 0', 'Hb_Value','Gender', 'Age_in_years', 'Image_Path'], errors='ignore')
#print(eyes_paths_df)

general_info = pd.read_excel('/content/anemia_data/Anemia_dataset.xlsx', sheet_name=0)
general_info = general_info.drop(columns=['Unnamed: 0', 'index', 'Total Serial Number', 'S No.', 'Unique ID'], errors='ignore')

#merging tables with general data about patients with df of images of eyes
eyes_paths_df.rename(columns = {'Blood_Sample_ID':'Blood Sample ID'}, inplace=True)
patients_df = pd.merge(general_info, eyes_paths_df, on='Blood Sample ID', how='outer')
patients_df.rename(columns = {'Haemoglobin (in mg/dl)':'Hemoglobin'}, inplace=True)

# Drop rows where 'Hemoglobin' is NaN
patients_df = patients_df.dropna(subset=['Left_Eye_Images'])
patients_df.reset_index(drop=True, inplace=True)

import numpy as np
mask_path = '/content/anemia_data/mask_eyes'

def get_masked_image(img_name):
  img = Image.open(os.path.join(eyes_path, img_name )).rotate(-90, expand=True)
  mask = Image.open(os.path.join(mask_path, img_name.replace('.jpg', '.png')))
  image_array = np.array(img)
  mask_array = np.array(mask)
  mask_array = mask_array / 255 # This makes mask values either 0 or 1
  masked_image_array = image_array * mask_array
  return masked_image_array.astype(np.uint8)

#filtering out the patients for which the mask doesnt exist
mask_exists = patients_df['Left_Eye_Images'].apply(
    lambda img_name: os.path.exists(os.path.join(mask_path, img_name.replace('.jpg', '.png')))
)

patients_df = patients_df[mask_exists]
patients_df.reset_index(drop=True, inplace=True)

patients_df

In [ ]:
eyes_path = '/content/anemia_data/Images_left_eye'
mask_path = '/content/anemia_data/mask_eyes'


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# Function to crop the masked conjunctiva image
def crop_masked_image(masked_image_array):
    # Convert to grayscale to identify non-black pixels
    gray = np.any(masked_image_array > 0, axis=-1)
    # Find rows and columns with non-zero values
    rows = np.where(np.any(gray, axis=1))[0]
    cols = np.where(np.any(gray, axis=0))[0]

    # If no non-black pixels found, return original (to avoid errors)
    if rows.size == 0 or cols.size == 0:
        return masked_image_array

    # Determine the cropping boundaries
    row_start, row_end = rows[0], rows[-1] + 1
    col_start, col_end = cols[0], cols[-1] + 1

    # Crop the image
    return masked_image_array[row_start:row_end, col_start:col_end]

def get_masked_image(img_name, image=None):
    """
    Apply the mask to an image. If no image is passed, the original image is loaded from disk.

    Args:
        img_name (str): The name of the image file.
        image (PIL.Image.Image, optional): An image to apply the mask to. If None, the original image is used.

    Returns:
        np.ndarray: The masked image as a NumPy array.
    """
    if image is None:
        image = Image.open(os.path.join(eyes_path, img_name)).rotate(-90, expand=True)

    # Load the mask corresponding to the image
    mask = Image.open(os.path.join(mask_path, img_name.replace('.jpg', '.png')))

    # Convert both image and mask to NumPy arrays
    image_array = np.array(image)
    mask_array = np.array(mask) / 255  # Normalize mask to [0, 1]

    # Apply the mask to the image
    masked_image_array = image_array * mask_array
    return masked_image_array.astype(np.uint8)


# Function to apply the mask and crop
def masked_and_cropped_image(img_name):
    masked_img = get_masked_image(img_name)  # Apply mask
    cropped_img = crop_masked_image(masked_img)  # Crop the image
    return cropped_img

# Sort the DataFrame by Hemoglobin
sorted_df = patients_df.sort_values(by='Hemoglobin')

# Function to display every 10th conjunctiva image (original and color corrected)
def display_every_10th_conjunctiva(sorted_df, eyes_path):
    # Extract every 10th image
    every_10th = sorted_df.iloc[::10]

    for i, row in every_10th.iterrows():
        img_name = row['Left_Eye_Images']
        hemoglobin_level = row['Hemoglobin']

        # Original masked and cropped conjunctiva
        original_cropped = masked_and_cropped_image(img_name)

        # Load the image for color correction
        img = Image.open(os.path.join(eyes_path, img_name)).rotate(-90, expand=True)

        # Get the transformation matrix (M) for the current image
        #M = np.array(df.loc[df['Left_Eye_Images'] == img_name, 'matrix_transform'].values[0])

        # Calculate M[1]^-1 (normalization matrix)
        #first_img_name = patients_df.iloc[0]['Left_Eye_Images']
        #first_M = np.array(df.loc[df['Left_Eye_Images'] == first_img_name, 'matrix_transform'].values[0])
        #M_inv_1 = np.diag([1,1,1])#np.linalg.inv(first_M)

        # Apply the color correction
        #corrected_img = apply_color_correction(img, M, M_inv_1)

        # Mask and crop the corrected image
        corrected_cropped = masked_and_cropped_image(img_name)
        #
        #masked_img = masked_image(img_name)  # Apply mask
      #cropped_img = crop_masked_image(masked_img)


        # Plot original and corrected images
        plt.figure(figsize=(12, 6))

        # Original Image
        plt.subplot(1, 2, 1)
        plt.imshow(original_cropped)
        plt.axis('off')
        plt.title(f"Original Conjunctiva\nHemoglobin: {hemoglobin_level:.2f}")

        # Corrected Image
        plt.subplot(1, 2, 2)
        plt.imshow(corrected_cropped)
        plt.axis('off')
        plt.title(f"Corrected Conjunctiva\nHemoglobin: {hemoglobin_level:.2f}")

        # Show the images
        plt.show()

# Call the function to display the conjunctiva images
#display_every_10th_conjunctiva(sorted_df, eyes_path)


In [ ]:
for i, image_name in enumerate(sorted_df['Left_Eye_Images']):
  plt.figure(figsize=(6, 3))
  original_cropped = masked_and_cropped_image(image_name)
  plt.imshow(original_cropped)
  plt.axis('off')
  if i ==10: break

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import cv2

# Image size and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Define directories
masked_images_dir = "masked_images"
os.makedirs(masked_images_dir, exist_ok=True)

# Define the padding and resizing function
def pad_and_resize(image, target_size=(224, 224)):
    # Get original dimensions
    h, w, c = image.shape
    # Calculate padding
    max_dim = max(h, w)
    top_pad = (max_dim - h) // 2
    bottom_pad = max_dim - h - top_pad
    left_pad = (max_dim - w) // 2
    right_pad = max_dim - w - left_pad
    # Apply padding
    padded_image = np.pad(
        image,
        ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)),
        mode='constant',
        constant_values=0  # Black padding
    )
    # Resize to target size
    resized_image = cv2.resize(padded_image, target_size)
    return resized_image

# Preprocess and save all images
masked_image_paths = []

for idx, row in patients_df.iterrows():
    original_path = row['Left_Eye_Images']
    masked_image = masked_and_cropped_image(original_path)  # Apply your masking function

    # Pad and resize the masked image
    processed_image = pad_and_resize(masked_image, target_size=IMAGE_SIZE)

    # Save the processed image
    new_filename = os.path.join(masked_images_dir, f"masked_image_{idx}.jpg")
    Image.fromarray((processed_image).astype('uint8')).save(new_filename)  # Save as 8-bit image
    masked_image_paths.append(new_filename)

# Add the new paths to the dataframe
patients_df['Masked_Images'] = masked_image_paths
print(masked_image_paths[1])

# Split the dataframe into train, validation, and test sets
train_df, temp_df = train_test_split(patients_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescaling for validation and test sets (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Masked_Images',
    y_col='Hemoglobin',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=True
)

# Validation generator
val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=None,
    x_col='Masked_Images',
    y_col='Hemoglobin',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=False
)

# Test generator
test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='Masked_Images',
    y_col='Hemoglobin',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=False
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Normalization, RandomFlip, RandomContrast, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda

def create_model():
    input_layer = Input(shape=(224, 224, 3))

    # Preprocessing layers
    x = Lambda(lambda t: tf.cast(t, tf.float32), name="cast_to_float32")(input_layer)  # Cast input to float32
    x = Normalization()(x)              # Normalize pixel values
    x = RandomFlip("horizontal")(x)     # Data augmentation: random horizontal flip
    x = RandomContrast(0.2)(x)          # Data augmentation: random contrast

    # ResNet50 backbone
    resnet_base = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    resnet_base.trainable = False       # Freeze ResNet50 backbone
    x = resnet_base(x)

    # Dropout layer
    x = Dropout(0.5)(x)

    # Flatten and regression head
    x = Flatten()(x)
    output = Dense(1, activation=None, name="regression_head")(x)  # Single output for regression

    # Compile the model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",  # Mean squared error for regression
        metrics=["mae"]  # Monitor mean absolute error
    )

    return model

model = create_model()
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_to_float32 (Lambda)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ normalization (Normalization)        │ (None, 224, 224, 3)         │               7 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip (RandomFlip)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast (RandomContrast)     │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 7, 7, 2048)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ regression_head (Dense)              │ (None, 1)                   │         100,353 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,688,072 (90.36 MB)

 Trainable params: 100,353 (392.00 KB)

 Non-trainable params: 23,587,719 (89.98 MB)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("best_model.h5.keras", save_best_only=True, monitor="val_loss"),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    ]
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 68.0137 - mae: 7.0481 - val_loss: 38.6406 - val_mae: 5.9811
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 248ms/step - loss: 23.7405 - mae: 4.1478 - val_loss: 7.3760 - val_mae: 2.2754
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 219ms/step - loss: 10.2336 - mae: 2.6276 - val_loss: 3.3666 - val_mae: 1.4237
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - loss: 7.3053 - mae: 2.3207 - val_loss: 10.2109 - val_mae: 2.8051
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 221ms/step - loss: 7.8859 - mae: 2.2549 - val_loss: 2.9589 - val_mae: 1.3797
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - loss: 5.4296 - mae: 1.7938 - val_loss: 3.5053 - val_mae: 1.4652
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - loss: 5.2764 - mae: 1.8742 - val_loss: 4.3541 - val_mae: 1.7238
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - loss: 7.2684 - mae: 2.2168 - val_loss: 5.1300 - val_mae: 1.9036
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 220ms/step - loss: 5.4968 - mae: 1.8688 - v

In [ ]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(test_generator)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 4.0812 - mae: 1.7207
Test Loss (MSE): 3.936042547225952
Test Mean Absolute Error (MAE): 1.6791688203811646


In [ ]:
history_continued = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,  # Additional epochs
    initial_epoch=history.epoch[-1] + 1,  # Start from the last epoch
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("best_model_continued.h5.keras", save_best_only=True, monitor="val_loss"),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    ]
)


Epoch 15/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 266ms/step - loss: 6.5815 - mae: 2.1312 - val_loss: 4.9139 - val_mae: 1.8507
Epoch 16/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - loss: 7.8609 - mae: 2.2699 - val_loss: 15.1049 - val_mae: 3.4967
Epoch 17/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step - loss: 8.4237 - mae: 2.4260 - val_loss: 7.2765 - val_mae: 2.3374
Epoch 18/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 235ms/step - loss: 6.7968 - mae: 2.0327 - val_loss: 3.2516 - val_mae: 1.4406
Epoch 19/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 5.5166 - mae: 1.8486 - val_loss: 6.6325 - val_mae: 2.1290
Epoch 20/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - loss: 7.1635 - mae: 2.2291 - val_loss: 3.4839 - val_mae: 1.4667


In [ ]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(test_generator)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 4.8032 - mae: 1.8045
Test Loss (MSE): 4.618839740753174
Test Mean Absolute Error (MAE): 1.7784041166305542


In [ ]:
import os
import zipfile

# List all files ending with .keras
keras_files = [file for file in os.listdir() if file.endswith(".keras")]

# Create a zip archive of all .keras files
with zipfile.ZipFile("models_keras.zip", "w") as zipf:
    for keras_file in keras_files:
        zipf.write(keras_file)

print("All models saved in models_keras.zip")


All models saved in models_keras.zip
